# Average speeds across entire trip

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import dask.dataframe as dd
import pandas as pd

from segment_speed_utils import helpers, sched_rt_utils
from segment_speed_utils.project_vars import (analysis_date, SEGMENT_GCS, 
                                              CONFIG_PATH, PROJECT_CRS)

In [2]:
STOP_SEG_DICT = helpers.get_parameters(CONFIG_PATH, "stop_segments")

INPUT_FILE = f'{STOP_SEG_DICT["stage2"]}_{analysis_date}'
INPUT_FILE

'vp_stop_segment_2023-05-17'

In [3]:
#test_key = "00accf770009aafd5dc103ff2eeddb37"
#test_trip = "t_1995375_b_33395_tn_0"

In [10]:
ddf = dd.read_parquet(
    f"{SEGMENT_GCS}vp_sjoin/{INPUT_FILE}", 
)

In [12]:
len(ddf.index)

27188771

In [ ]:
operators = dd.read_parquet(
    f"{SEGMENT_GCS}{INPUT_FILE}", 
    columns = ["gtfs_dataset_key"]
).gtfs_dataset_key.unique().compute().tolist()

In [ ]:
subset_operators = operators[:2]
subset_operators

In [ ]:
ddf = dd.read_parquet(
    f"{SEGMENT_GCS}{INPUT_FILE}", 
    filters = [[("gtfs_dataset_key", "in", subset_operators)]],
    columns = ["vp_idx"]
)

In [ ]:
trip_cols = ["gtfs_dataset_key", "trip_id"]
hour_min_cols = ["hour", "minute"]

## Pings per minute for service hours

In [ ]:
ddf = ddf.repartition(npartitions=5)

ddf = ddf.assign(
    minute = ddf.location_timestamp_local.dt.minute
)

In [ ]:
ddf.dtypes

In [ ]:
num_vp_pings = (ddf.groupby(trip_cols + hour_min_cols, observed=True)
                ["location_timestamp_local"]
                .count()
                .dropna()
                .reset_index()
                .rename(columns = {"location_timestamp_local": "num_pings"})
               )

In [ ]:
num_vp_pings = num_vp_pings.assign(
        atleast2 = num_vp_pings.apply(
            lambda x: 1 if x.num_pings >= 2
            else 0, axis=1, meta=('atleast2', 'int8'))
    )    

In [ ]:
vp_pings = (num_vp_pings.groupby(trip_cols)
            .agg({
               "hour": "size",
               "atleast2": "sum"})
            .dropna()
            .reset_index()
           ).rename(columns = {
            "hour": "trip_min_elapsed"})

In [ ]:
vp_pings = vp_pings.persist()

In [ ]:
vp_pings.compute()

## Triangulate vp for lengths

In [ ]:
from dask import delayed, compute

df_orig = delayed(pd.read_parquet)(
    f"{SEGMENT_GCS}{INPUT_FILE}", 
    filters = [[("gtfs_dataset_key", "in", subset_operators)]],
    columns = ["gtfs_dataset_key", "trip_id", "vp_idx", 
               "location_timestamp_local"]
)

In [ ]:
df_orig2 = df_orig.groupby(trip_cols, observed=True).agg(
    {"vp_idx": list}).reset_index()

In [ ]:
df_orig3 = compute(df_orig2)[0]

In [ ]:
import numpy as np

def count_vp_and_get_every_10_min(my_list: list):
    vp_idx_arr = np.asarray(my_list)
    subset_arr = vp_idx_arr[::30]
    
    if len(subset_arr) < 2:
        subset_arr = vp_idx_arr[:15]
    
    return list(subset_arr)

In [ ]:
df_orig3 = df_orig3.assign(
    vp_idx2 = df_orig3.apply(lambda x: 
                             count_vp_and_get_every_10_min(x.vp_idx), 
                             axis=1)
)

In [ ]:
keep_subset_vp = list(df_orig3.vp_idx2.explode())

In [ ]:
ddf = dd.read_parquet(
    f"{SEGMENT_GCS}{INPUT_FILE}", 
    filters = [[("gtfs_dataset_key", "in", subset_operators), 
               ("vp_idx", "in", keep_subset_vp)]],
    columns = ["gtfs_dataset_key", "trip_id", 
               "location_timestamp_local", 
               "x", "y",
               "vp_idx"]
).compute()

In [ ]:
trip_grouping_cols = ["shape_array_key"]
crosswalk = sched_rt_utils.crosswalk_scheduled_trip_grouping_with_rt_key(
        analysis_date, 
        ["feed_key", "trip_id"] + trip_grouping_cols
    )

In [ ]:
vp_with_shape = dd.merge(
    ddf,
    crosswalk,
    on = ["gtfs_dataset_key", "trip_id"],
    how = "inner"
).drop_duplicates()

In [ ]:
subset_shapes = vp_with_shape.shape_array_key.unique().tolist()

In [ ]:
shapes = helpers.import_scheduled_shapes(
    analysis_date,
    columns = ["shape_array_key", "geometry"],
    filters = [[("shape_array_key", "in", subset_shapes)]],
    get_pandas = True,
    crs = PROJECT_CRS
)

In [ ]:
from segment_speed_utils import wrangle_shapes
import dask_geopandas as dg
import geopandas as gpd

In [ ]:
triangulated_vp_with_shape = dd.from_pandas(vp_with_shape, npartitions=1)


In [ ]:
triangulated_vp_with_shape["geometry"] = dg.points_from_xy(
    triangulated_vp_with_shape, "x", "y")

# Refer to the geometry column by name
vp_gddf = dg.from_dask_dataframe(
    triangulated_vp_with_shape, 
    geometry="geometry"
).set_crs("EPSG:4326").to_crs(PROJECT_CRS)

In [ ]:
vp_gddf2 = dd.merge(
    vp_gddf,
    shapes,
    on = "shape_array_key",
    how = "inner"
)

In [ ]:
vp_geoseries = gpd.GeoSeries(vp_gddf2.geometry_x.compute())
shape_geoseries = gpd.GeoSeries(vp_gddf2.geometry_y.compute())

In [ ]:
vp_geoseries_buff = vp_geoseries.buffer(25)

In [ ]:
intersecting_line = vp_geoseries_buff.intersection(shape_geoseries)

In [ ]:
gdf = vp_gddf2.compute()
gdf["area_of_intersection"] = intersecting_line

In [ ]:
gdf.area_of_intersection.is_empty

In [ ]:
gdf = gdf.set_geometry("area_of_intersection").set_crs(PROJECT_CRS)
gdf2 = gdf[~gdf.set_geometry("area_of_intersection").is_empty]

In [ ]:
test_key = "00accf770009aafd5dc103ff2eeddb37"
test_trip = "t_1995375_b_33395_tn_0"
gdf2[gdf2.trip_id == "t_1995375_b_33395_tn_0"]

In [ ]:
#points_plot = vp_gddf[
#    vp_gddf.trip_id == "t_1995375_b_33395_tn_0"].compute()[
#    ["vp_idx", "shape_array_key", "geometry"]]


points_plot = pd.read_parquet(
    f"{SEGMENT_GCS}{INPUT_FILE}", 
    filters = [[("gtfs_dataset_key", "in", subset_operators), 
               ("trip_id", "==", "t_1995375_b_33395_tn_0")]],
    columns = ["gtfs_dataset_key", "trip_id", 
               #"location_timestamp_local", 
               "x", "y",
               "vp_idx"]
)#.compute()

In [ ]:
crosswalk[crosswalk.trip_id=="t_1995375_b_33395_tn_0"].shape_array_key.iloc[0]

In [ ]:
points_plot["geometry"] = gpd.points_from_xy(points_plot.x, points_plot.y, 
                                             crs="EPSG:4326")

In [ ]:
points_plot = points_plot.set_geometry("geometry").to_crs(PROJECT_CRS)

In [ ]:
shape_layer = shapes[shapes.shape_array_key==points_plot.shape_array_key.iloc[0]]

In [ ]:
import folium

In [ ]:
m = points_plot.explore("vp_idx", tiles = "CartoDB Positron", )
m = shape_layer.explore(m=m, color="red", name="shape")
folium.LayerControl().add_to(m)
m

In [ ]:
sample2 = pd.merge(
    sample,
    shapes,
    on = "shape_array_key",
    how = "inner"
)

In [ ]:
one_shape_geom = sample2.geometry_y.iloc[0]

In [ ]:
one_shape_geom.unary_union

In [ ]:
import shapely

In [ ]:
sample2 = sample2.assign(
    shape_meters = sample2.apply(
        lambda x: x.geometry_y.unary_union.project(x.geometry_x), 
        axis=1)
)

In [ ]:
shapes[shapes.shape_array_key=="70f010e0dba18191937ed4b5bea42e8a"].explore(
    tiles = "CartoDB Positron")

In [ ]:
one_shape = shapes[
    shapes.shape_array_key=="70f010e0dba18191937ed4b5bea42e8a"].geometry.iloc[0]

In [ ]:
test = wrangle_shapes.project_list_of_coords(one_shape, [], use_shapely_coords=True)

In [ ]:
test

In [ ]:
df2 = wrangle_shapes.linear_reference_vp_against_segment(
    triangulated_vp_with_shape,
    shapes,
    segment_identifier_cols = ["shape_array_key"]
)

In [ ]:
df2.compute()